In [ ]:
from keras.models import load_model
from array import array
from glob import glob
import pandas as pd
import collections
import uproot
import ROOT
import sys
from os import environ, path, mkdir, listdir
import multiprocessing
import subprocess
import h5py
environ['KMP_DUPLICATE_LIB_OK']='True'

def getGuess(df, index):
    try:
        prob_sig = df.loc[index, 'prob_sig']
    except:
        prob_sig = -999
    return prob_sig

def build_filelist(input_dir):
    filelist = collections.defaultdict(list)
    filelist['all'] = [fname for fname in glob('{}/*.csv'.format(input_dir))]
    return filelist

model = load_model('/localdata/Athar/models/{}.hdf5'
                   .format('model_name'))
all_data = pd.HDFStore('/localdata/Athar/datasets/preprocessed_dataset.h5.h5') 
if not path.isdir('/localdata/Athar/output_files'):
    mkdir('/localdata/Athar/output_files')

filelist = build_filelist('/localdata/Athar/input_files')
for _, files in filelist.items():
    for ifile in files:
        print ('Processing file: {}'.format(ifile))
        fname = ifile
        MakeCsvDataFrame = ROOT.RDF.MakeCsvDataFrame
        csv_file = MakeCsvDataFrame(ifile)
        input_dict = csv_file.AsNumpy()
        output_file = pd.DataFrame.from_dict(input_dict)
        data = all_data['nominal']
        print('Sample' + fname)
        sample = data[(data['sample_names'] == fname)]
        to_classify = sample[['pxB1', 'pyB1', 'pzB1', 'eB1', 'pxB2', 'pyB2', 'pzB2', 'eB2', 'pxJ1', 'pyJ1', 'pzJ1', 'eJ1',
              'pxJ2', 'pyJ2', 'pzJ2', 'eJ2', 'pxL1', 'pyL1', 'pzL1', 'eL1', 'pxN1', 'pyN1', 'pzN1', 'eN1',
              'pxH', 'pyH', 'pzH', 'eH', 'pxt11', 'pyt11', 'pzt11', 'et11', 'pxt12', 'pyt12', 'pzt12',
              'et12', 'pxt21', 'pyt21', 'pzt21', 'et21', 'pxt22', 'pyt22', 'pzt22', 'et22', 'pxW1', 'pyW1', 
              'pzW1', 'eW1', 'pxW2', 'pyW2', 'pzW2', 'eW2', 'm_B1', 'pt_B1', 'eta_B1', 'phi_B1', 'm_B2', 
              'pt_B2', 'eta_B2', 'phi_B2', 'm_J1', 'pt_J1', 'eta_J1', 'phi_J1', 'm_J2', 'pt_J2', 'eta_J2', 
              'phi_J2', 'm_L1', 'pt_L1', 'eta_L1', 'phi_L1', 'm_N1', 'pt_N1', 'eta_N1', 'phi_N1', 'm_H', 
              'pt_H', 'eta_H', 'phi_H', 'm_t11', 'pt_t11', 'eta_t11', 'phi_t11', 'm_t12', 'pt_t12', 
              'eta_t12', 'phi_t12', 'm_t21', 'pt_t21', 'eta_t21', 'phi_t21', 'm_t22', 'pt_t22', 'eta_t22', 
              'phi_t22', 'm_W1', 'pt_W1', 'eta_W1', 'phi_W1', 'm_W2', 'pt_W2', 'eta_W2', 'phi_W2'
             ]]
        
        print(to_classify.shape)
        guesses = model.predict(to_classify.values, verbose=True)
        out = data.copy()
        output_file['prob_sig'] = guesses[:,0]
        print(output_file)
        out_name = ifile.split('/')[-1].replace('.csv', '_classified.csv')
        output_file.to_csv('/localdata/Athar/output_files/{}'.format(out_name), index=False)